In [332]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax, expit

In [333]:
from loaders import clean_text_for_pattern_letter, get_pattern_for_letter_without_collision
from temp_text import template_str
input_str = clean_text_for_pattern_letter(template_str, out_len=500)
print(len(input_str))

letter_pattern = get_pattern_for_letter_without_collision(size=128, pattern_size=4, dump_cache=True)
print(len(letter_pattern))

len(letters)=32
500
a 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
b 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
c 4 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [334]:
samples_l = []

for inp in input_str:
    pattern = letter_pattern[inp]
    samples_l.append(pattern)

samples = np.array(samples_l)
print(samples.shape)

(500, 128)


In [335]:
def swish(x):
    return x * expit(x)

In [336]:
def normalize(x):
    
    mean = x.mean(axis = 0)
    std = x.std(axis = 0)
    
    return (x - mean) / np.sqrt(std + 1e-5)

In [337]:
def softmax_grad(err_grad):
    # Reshape the 1-d softmax to 2-d so that np.dot will do the matrix multiplication
    s = err_grad.reshape(-1,1)
    return np.diagflat(s) - np.dot(s, s.T)

In [338]:
def z_fd(z, err_grad):
    sfm = np.reshape(z, (1, -1))
    grad = np.reshape(err_grad, (1, -1))

    d_softmax = (sfm * np.identity(sfm.size) - sfm.transpose() @ sfm)

    return (grad @ d_softmax).ravel()

In [339]:
def show_inp_mat(M):
    print(M.shape)
    plt.matshow(M.T)
    plt.colorbar()
    plt.show()

In [340]:
# train
col_size = 32

samples_size = 10
samples = samples[:samples_size]

mu = 0.0
sigma = 1.0

I = np.zeros((samples.shape[1], col_size))
O = np.zeros(I.shape)
W = np.random.normal(mu, sigma, (I.shape[0], I.shape[1], I.size))
E = np.zeros(I.shape[0])

dW = np.zeros(W.shape)

af_pow = 2

I = np.random.rand(I.shape[0], I.shape[1]) * 1e-3
I = np.power(I, af_pow)
I = softmax(I)
# I = normalize(I)

epoch_cnt = 10
eps0 = 0.02

for epoch_idx in range(epoch_cnt):
    eps = (epoch_cnt - epoch_idx) * eps0

    batch_E = []

    print('W=',W.shape, W.max(), W.min(), np.mean(W))

    for i_idx in range(samples.shape[0] - 1):
    # for i_idx in range(1):
        next_idx = i_idx + 1

        IV = samples[i_idx]
        OVR = samples[next_idx]

        # infer
        I = I * IV[np.newaxis].T

        # show_inp_mat(I)

        O = np.tensordot(W, I.reshape(I.size), 1)

        # show_inp_mat(O)

        O = np.power(O, af_pow)
        O = softmax(O)
        # O = normalize(O)
        # O /= np.linalg.norm(O, ord=2)

        # show_inp_mat(O)

        OV = np.sum(O, axis=1)
        # show_inp_mat(OV[np.newaxis].T)

        E = (OVR - OV) ** 2
        # show_inp_mat(E[np.newaxis].T)
        print(E.sum())
        batch_E.append(E.sum())

        # bp
        EO = E[np.newaxis].T * np.ones(O.shape[1])
        # show_inp_mat(EO)

        # EO = softmax_grad(EO)
        EO = z_fd(O, EO).reshape(O.shape)
        # show_inp_mat(EO)

        EO = af_pow * np.power(EO, af_pow-1)
        # show_inp_mat(EO)

        dW += EO.reshape(EO.shape[0], EO.shape[1], 1) * I.reshape(I.size).reshape(1, 1, I.size)
        # print(EO.reshape(EO.shape[0], EO.shape[1], 1)[0][0])
        # print(I.reshape(I.size).reshape(1, 1, I.size)[0][0].size)
        print(dW.shape, dW.max(), dW.min())

        # reccurention
        I = np.copy(O)

    # update
    print('W(before_update)=',W.shape, W.max(), W.min(), np.mean(W))
    W -= dW * eps

    err = np.mean(np.array(batch_E))
    print(f'{err=}')
    print('W(updated)=',W.shape, W.max(), W.min(), np.mean(W))


W= (128, 32, 4096) 5.543143330204086 -5.305571535094439 0.00016117659079396674
3.9453125158604916
(128, 32, 4096) 1.1368599372245476e-07 -3.6674357142816147e-09
3.9453124123940784
(128, 32, 4096) 1.1369952860258186e-07 -3.6678297400331536e-09
3.945312467357349
(128, 32, 4096) 1.1369952860258186e-07 -3.668030830892251e-09
3.9453124292774753
(128, 32, 4096) 1.1369952860258186e-07 -3.668030830892251e-09
3.9453125416956416
(128, 32, 4096) 1.1369952860258186e-07 -3.668030830892251e-09
3.945312428100542


KeyboardInterrupt: 